# Results

In [32]:
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [35]:
def get_runs(entity, project): #(entity, project, log_keys=None, filters=None):
    api = wandb.Api(timeout=20)
    runs = api.runs(f"{entity}/{project}", filters={"created_at": {"$gt": "2025-02-23T20"}})
    print(f"Found {len(runs)} runs.")
    history = []
    for run in runs:
        #if log_keys:
        #    for log in run.scan_history(keys=log_keys):
        #        log["config"] = dict(run.config)
        #        history.append(log)
        #else:
        history.append({"config": dict(run.config), **dict(run.summary)})

    df = pd.json_normalize(history)
    return runs, df

In [36]:
runs, df = get_runs("jccamml","L65-project")

Found 32 runs.


In [37]:
df_full = df[df['_step'] > 8000]

In [27]:
[c for c in df_full.columns if "teacher" in c]

['config.hint_teacher_forcing']

In [15]:
df_full['config.algorithms']

0              [heapsort]
16       [insertion_sort]
17       [insertion_sort]
35       [insertion_sort]
36             [heapsort]
37             [heapsort]
38    [activity_selector]
39             [heapsort]
40       [insertion_sort]
41    [activity_selector]
42    [activity_selector]
43       [insertion_sort]
44             [heapsort]
45    [activity_selector]
46       [insertion_sort]
47             [heapsort]
48    [activity_selector]
49       [insertion_sort]
50             [heapsort]
51       [insertion_sort]
52    [activity_selector]
Name: config.algorithms, dtype: object

In [22]:
df_full['config.algorithms'] = df_full['config.algorithms'].apply(lambda x: x[0])   

/tmp/ipykernel_23478/205732895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['config.algorithms'] = df_full['config.algorithms'].apply(lambda x: x[0])


In [46]:
df_full.pivot_table(index=['config.processor_type', 'config.hint_teacher_forcing', 'config.mpnn_processor_aggregator'], values=['insertion_sort_test_score', 'activity_selector_test_score'], aggfunc=np.mean)

/tmp/ipykernel_23478/2090578102.py:1: FutureWarning: The provided callable <function mean at 0x72afa8101da0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_full.pivot_table(index=['config.processor_type', 'config.hint_teacher_forcing', 'config.mpnn_processor_aggregator'], values=['insertion_sort_test_score', 'activity_selector_test_score'], aggfunc=np.mean)


activity_selector_test_score  \
config.processor_type      config.hint_teacher_forcing config.mpnn_processor_aggregator                                 
differential_mpnn          0.0                         max                                                   0.913444   
                                                       sum                                                   0.663537   
differential_mpnn2         0.0                         max                                                   0.956522   
                                                       sum                                                   0.821092   
differential_mpnn_identity 0.0                         max                                                   0.920578   
                                                       sum                                                   0.211495   
differential_mpnn_maxmax   0.0                         max                                                   0.889680   
differential_mpnn_msgdiff  0.0                         max                                                   0.935543   
mpnn                       0.0                         max                                                   0.942238   
                                                       sum                                                   0.649895   
                           0.5                         max                                                   0.956679   
                                                       sum                                                   0.238280   

                                                                                         insertion_sort_test_score  
config.processor_type      config.hint_teacher_forcing config.mpnn_processor_aggregator                             
differential_mpnn          0.0                         max                                                0.023926  
                                                       sum                                                0.869141  
differential_mpnn2         0.0                         max                                                0.291016  
                                                       sum                                                0.029297  
differential_mpnn_identity 0.0                         max                                                0.578125  
                                                       sum                                                0.128906  
differential_mpnn_maxmax   0.0                         max                                                0.581055  
differential_mpnn_msgdiff  0.0                         max                                                     NaN  
mpnn                       0.0                         max                                                0.617188  
                                                       sum                                                0.329102  
                           0.5                         max                                                0.149414  
                                                       sum                                                0.050293

In [ ]:
df_full.pivot_table(index=['config.processor_type', 'config.hint_teacher_forcing'], values=['test_score_current_model_heapsort', 'insertion_sort_test_score', 'activity_selector_test_score'], aggfunc=np.mean)

KeyError: 'test_score_current_model_heapsort'

In [24]:
for algorithm in df_full['config.algorithms'].unique():
    print(algorithm)
    print(df_full[df_full['config.algorithms'] == algorithm][['config.processor_type', f'test_score_current_model_{algorithm}']].to_markdown())
    print()

heapsort
|    | config.processor_type      |   test_score_current_model_heapsort |
|---:|:---------------------------|------------------------------------:|
|  0 | differential_mpnn          |                           0.706055  |
| 36 | mpnn                       |                           0.368164  |
| 37 | differential_mpnn2         |                           0.287598  |
| 39 | differential_mpnn_identity |                           0.0454102 |
| 44 | differential_mpnn          |                           0.382812  |
| 47 | mpnn                       |                           0.120117  |
| 50 | differential_mpnn_msgdiff  |                           0.45459   |

insertion_sort
|    | config.processor_type      |   test_score_current_model_insertion_sort |
|---:|:---------------------------|------------------------------------------:|
| 16 | differential_mpnn          |                                 0.675293  |
| 17 | mpnn                       |                                 0

In [31]:
import jax 
jax.devices()

[CpuDevice(id=0)]

In [48]:
np.expand_dims(np.zeros((3,4,2)),2).shape

(3, 4, 1, 2)

In [50]:
np.mean(np.zeros((5,6,7,8)), axis=1).shape

(5, 7, 8)

In [15]:
import jax 
import numpy as np
import jax.numpy as jnp

In [48]:
def entropy_from_softmax_logits(logits):
  logZ = jax.scipy.special.logsumexp(logits, axis=-1)
  ps = jax.scipy.special.softmax(logits, axis=-1)
  #print(logZ.shape,(ps * logits).sum(axis=-1).shape)
  return logZ.mean() - (ps * logits).sum(axis=-1).mean()

In [30]:
entropy_from_softmax_logits(np.array([[0]* 1000 + [10]])) ##np.random.randn(100,100,100) * np.inf) #,[1.,1.,1.],[0,100.,0.]

(1,) (1,)


Array(0.47868252, dtype=float32)

In [16]:
# 1. Basic Case: Small logits where we can compute entropy manually
logits1 = jnp.array([[0.0, 0.0]])  # Uniform over two classes
expected_entropy1 = jnp.log(2)  # log(2) ≈ 0.693
print("Test 1:", entropy_from_softmax_logits(logits1), "Expected:", expected_entropy1)

# 2. Uniform distribution: Maximum entropy case
logits2 = jnp.array([[1.0, 1.0, 1.0]])  # 3 equal probabilities
expected_entropy2 = jnp.log(3)  # log(3) ≈ 1.099
print("Test 2:", entropy_from_softmax_logits(logits2), "Expected:", expected_entropy2)

# 3. One-class dominance: Near-zero entropy
logits3 = jnp.array([[10.0, -10.0]])  # Softmax will be ~ [1, 0]
expected_entropy3 = 0.0
print("Test 3:", entropy_from_softmax_logits(logits3), "Expected:", expected_entropy3)

# 4. Negative and positive logits
logits4 = jnp.array([[-2.0, 0.0, 2.0]])
entropy4 = entropy_from_softmax_logits(logits4)
print("Test 4:", entropy4)

# 5. Large-scale logits: Check numerical stability
logits5 = jnp.array([[1000.0, 1000.0, 1000.0]])  # Should still return log(3)
expected_entropy5 = jnp.log(3)
print("Test 5:", entropy_from_softmax_logits(logits5), "Expected:", expected_entropy5)

# 6. Large batch size
logits6 = jnp.ones((1000, 10))  # 1000 samples, uniform 10-class distribution
expected_entropy6 = jnp.log(10)
print("Test 6:", entropy_from_softmax_logits(logits6), "Expected:", expected_entropy6)

(1,) (1,)
Test 1: 0.6931472 Expected: 0.6931472
(1,) (1,)
Test 2: 1.0986123 Expected: 1.0986123
(1,) (1,)
Test 3: 0.0 Expected: 0.0
(1,) (1,)
Test 4: 0.44105744
(1,) (1,)
Test 5: 1.0986328 Expected: 1.0986123
(1000,) (1000,)
Test 6: 2.3025856 Expected: 2.3025851


In [58]:
tmp = None
for p in range(-1000,1000,100):
    for c in range(-1000,1000,100):
        logits = np.random.randn(100) * p + c
        if entropy_from_softmax_logits(logits) < 0:
            print(p,c,entropy_from_softmax_logits(logits))
            tmp = logits
            break

-800 -300 -0.00012207031


In [63]:
logits = tmp[:]
for i in range(100):
    entropy, grad = jax.value_and_grad(entropy_from_softmax_logits)(logits)
    print(f"{i}: {entropy}, {jnp.linalg.norm(grad)}")
    logits -= 0.01 * grad

0: -0.0001220703125, 0.00012207258259877563
1: -0.0001220703125, 0.00012207258259877563
2: -0.0001220703125, 0.00012207258259877563
3: -0.0001220703125, 0.00012207258259877563
4: -0.0001220703125, 0.00012207258259877563
5: -0.0001220703125, 0.00012207258259877563
6: -0.0001220703125, 0.00012207258259877563
7: -0.0001220703125, 0.00012207258259877563
8: -0.0001220703125, 0.00012207258259877563
9: -0.0001220703125, 0.00012207258259877563
10: -0.0001220703125, 0.00012207258259877563
11: -0.0001220703125, 0.00012207258259877563
12: -0.0001220703125, 0.00012207258259877563
13: -0.0001220703125, 0.00012207258259877563
14: -0.0001220703125, 0.00012207258259877563
15: -0.0001220703125, 0.00012207258259877563
16: -0.0001220703125, 0.00012207258259877563
17: -0.0001220703125, 0.00012207258259877563
18: -0.0001220703125, 0.00012207258259877563
19: -0.0001220703125, 0.00012207258259877563
20: -0.0001220703125, 0.00012207258259877563
21: -0.0001220703125, 0.00012207258259877563
22: -0.0001220703125

In [35]:
jax.scipy.special.logsumexp(logits , axis=-1)

Array([20.], dtype=float32)

In [38]:
(jax.scipy.special.softmax(logits , axis=-1) * logits).sum(axis=-1)

Array([19.999998], dtype=float32)

# Results Analysis

In [ ]:
import wandb
import yaml  # For saving the config

# Initialize API
api = wandb.Api()

# Define project details
entity = "your-entity"   # Change to your entity name
project = "your-project" # Change to your project name

# Fetch runs
runs = api.runs(f"{entity}/{project}")

# Define filtering criteria (modify as needed)
def filter_run(run):
    return (
        run.config.get("learning_rate", 0) > 0.01 and 
        run.summary.get("accuracy", 0) > 0.8
    )

# Extract configurations from matching runs
filtered_configs = {run.name: run.config for run in runs if filter_run(run)}

# Save to a config file
with open("filtered_configs.yaml", "w") as f:
    yaml.dump(filtered_configs, f)

print(f"Saved {len(filtered_configs)} filtered configs.")

In [ ]:
import wandb
# Initialize wandb API
api = wandb.Api()

# Define the project (replace with your actual project and entity)
entity = "jccamml"
project = "L65-project"

# Fetch runs
runs = api.runs(f"{entity}/{project}", {
        "created_at": {"$gt": "2025-03-23T01"}, # greater than
    })

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import sem



# Extract scores for all runs
all_scores = {}

score_field="charts/eval_avg_episodic_reward_new"
for run in runs:
    cfg = run.config
    summary = run.summary

    if 'kadane' not in cfg['algorithms'][0] or summary.get('num_steps',0) < 80000:
        continue
    print(run.name, cfg, summary)
    break